In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "FASTX",
    "BioSequences",
    "Kmers",
    "Graphs",
    "MetaGraphs",
    "SparseArrays",
    "ProgressMeter",
    "Distributions",
    "HiddenMarkovModels",
    "BioAlignments",
    "StatsBase",
    "Primes"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
reference_fastas = sort(filter(x -> occursin(r"\.fna$", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
reference_fasta = first(reference_fastas)

In [ ]:
filter(x -> occursin(reference_fasta, x) && occursin("10x", x) && occursin(r"\.fq\.gz$", x), readdir(genome_dir, join=true))

In [ ]:
fastq = first(filter(x -> occursin(reference_fasta, x) && occursin("10x", x) && occursin("filtlong", x) && occursin(r"\.fq\.gz$", x), readdir(genome_dir, join=true)))

In [ ]:
k = Mycelia.assess_dnamer_saturation([fastq], plot=false)
stranded_kmer_graph = Mycelia.build_stranded_kmer_graph(Kmers.DNAKmer{k}, collect(Mycelia.open_fastx(fastq)))
# 141.457417 seconds
@time updated_records = Mycelia.viterbi_maximum_likelihood_traversals(stranded_kmer_graph)

In [ ]:
k = Primes.nextprime(k + 1)
stranded_kmer_graph = Mycelia.build_stranded_kmer_graph(Kmers.DNAKmer{k}, updated_records)
@time updated_records = Mycelia.viterbi_maximum_likelihood_traversals(stranded_kmer_graph)

In [ ]:
k = Primes.nextprime(k + 1)
stranded_kmer_graph = Mycelia.build_stranded_kmer_graph(Kmers.DNAKmer{k}, updated_records)
@time updated_records = Mycelia.viterbi_maximum_likelihood_traversals(stranded_kmer_graph)

In [ ]:
k = Primes.nextprime(k + 1)
stranded_kmer_graph = Mycelia.build_stranded_kmer_graph(Kmers.DNAKmer{k}, updated_records)
@time updated_records = Mycelia.viterbi_maximum_likelihood_traversals(stranded_kmer_graph)

In [ ]:
temp_fasta = Mycelia.write_fasta(records = updated_records)

In [ ]:
# make my own kmer graph
assembly_graph = Mycelia.fastx_to_kmer_graph(Kmers.DNAKmer{k}, temp_fasta)

In [ ]:
gfa_file = temp_fasta * ".gfa"

In [ ]:
Mycelia.graph_to_gfa(graph=assembly_graph, outfile=temp_fasta * ".gfa")

In [ ]:
image = fastq * ".mycelia.gfa.jpg"

In [ ]:
# run(`chmod +x $(homedir())/software/bin/Bandage`)
run(`$(homedir())/software/bin/Bandage image $(gfa_file) $(image)`)

In [ ]:
run(``)

In [ ]:
Mycelia.ban

In [ ]:
# simplify it (if I still have that code)

In [ ]:
# write it to .gfa

In [ ]:
# pass .gfa into PGGB & VG

In [ ]:
# call variants of original against modified